In [47]:
!pip install tensorflow==2.3.0

     |████████████████████████████████| 320.4 MB 17 kB/s s eta 0:00:01
     |████████████████████████████████| 26.1 MB 35.7 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 2.0 MB/s  eta 0:00:01
     |████████████████████████████████| 6.0 MB 27.6 MB/s eta 0:00:01
     |████████████████████████████████| 459 kB 73.8 MB/s eta 0:00:01
     |████████████████████████████████| 781 kB 79.5 MB/s eta 0:00:01
     |████████████████████████████████| 4.9 MB 19.7 MB/s eta 0:00:01
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.1.0
    Uninstalling tensorflow-estimator-2.1.0:
      Successfully uninstalled tensorflow-estimator-2.1.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.1.1
    Uninstalling tensorboard-2.1.1:
      Successfully uninstalled tensorboard-2.1.1
  Attempting uninstall: scipy
    Found existing installation: scipy 1.5.3
    Uninstalling scipy-1.5.3:
      Successfully uninstalled

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow2_p36/bin/python -m pip install --upgrade pip' command.


In [3]:
import boto3
from tqdm import tqdm

In [4]:
bucket_name='captiongeneratorapp'
s3=boto3.resource('s3')
my_bucket=s3.Bucket(bucket_name)
my_bucket

s3.Bucket(name='captiongeneratorapp')

In [5]:
objs=[]
for my_bucket_object in tqdm(my_bucket.objects.filter(Prefix='Dataset/train2014/COCO_train2014_')):
    objs.append(my_bucket_object)

82783it [01:40, 821.67it/s]


In [6]:
len(objs)

82783

In [7]:
conn=boto3.client('s3')
conn

# Preprocess the image using InceptionV3

In [8]:
import tensorflow as tf
import numpy as np
import cv2
tf.__version__

'2.3.0'

In [9]:
image_model = tf.keras.applications.InceptionV3(include_top=True,
                                                weights='imagenet')
new_input = image_model.input
hidden_layer = image_model.layers[-2].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

In [22]:
def get_image(content):
    image = np.frombuffer(content, np.int8)
    cv_compatible_image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    resized_image = cv2.resize(cv_compatible_image, (299,299))
    img = tf.keras.applications.inception_v3.preprocess_input(resized_image)
    return img

In [19]:
def get_image(obj):
#     print(obj)
    with tf.compat.v1.Session() as sess:
        file_obj=conn.get_object(Bucket=bucket_name,Key=obj.eval(session=sess))
    content=file_obj['Body'].read()
    image = np.frombuffer(content, np.int8)
    cv_compatible_image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    resized_image = cv2.resize(cv_compatible_image, (299,299))
    img = tf.keras.applications.inception_v3.preprocess_input(resized_image)
    return img

In [11]:
all_keys=[obj.key for obj in objs]
len(all_keys),all_keys[:2]

(82783,
 ['Dataset/train2014/COCO_train2014_000000000009.jpg',
  'Dataset/train2014/COCO_train2014_000000000025.jpg'])

In [20]:
image_dataset = tf.data.Dataset.from_tensor_slices(all_keys)
image_dataset = image_dataset.map(
  get_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(8)

InvalidArgumentError: in user code:

    <ipython-input-19-8866bba98cfe>:4 get_image  *
        file_obj=conn.get_object(Bucket=bucket_name,Key=obj.eval(session=sess))
    /home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/tensorflow/python/framework/ops.py:913 eval  **
        return _eval_using_default_session(self, feed_dict, self.graph, session)
    /home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/tensorflow/python/framework/ops.py:5512 _eval_using_default_session
        return session.run(tensors, feed_dict)
    /home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/tensorflow/python/client/session.py:958 run
        run_metadata_ptr)
    /home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/tensorflow/python/client/session.py:1181 _run
        feed_dict_tensor, options, run_metadata)
    /home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/tensorflow/python/client/session.

In [49]:
tf.python.framework.ops.Tensor.eval

'2.1.3'

In [29]:
for obj in tqdm(objs[41877:]):
    file_obj=conn.get_object(Bucket=bucket_name,Key=obj.key)
    content=file_obj['Body'].read()
    image=get_image(content)
    features=image_features_extract_model(np.array([image]))
    np.save('train2014npy/'+obj.key.split('/')[-1],features)

100%|██████████| 40906/40906 [1:46:10<00:00,  6.42it/s]  


In [29]:
'train2014npy/'+obj.key.split('/')[-1]

'train2014npy/COCO_train2014_000000000009.jpg'

In [ ]:
1+2

In [24]:
obj

s3.ObjectSummary(bucket_name='captiongeneratorapp', key='Dataset/train2014/COCO_train2014_000000338474.jpg')

In [42]:
import os
lt=os.listdir('train2014npy')

In [43]:
len(lt),len(set(lt)),lt[0]

(82783, 82783, 'COCO_train2014_000000297632.jpg.npy')

In [27]:
objs[48176]

s3.ObjectSummary(bucket_name='captiongeneratorapp', key='Dataset/train2014/COCO_train2014_000000338474.jpg')

In [28]:
lt[-1]

'COCO_train2014_000000161782.jpg.npy'